In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:

# Cargar la imagen 
image = cv2.imread('assets/coronaCircular3d2.png')
if image is None:
    raise ValueError("Imagen no encontrada. Verifica la ruta.")

# --- Preprocesamiento: Conversión a gris y binario ---
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

# Guardar imágenes
cv2.imwrite('output/gris.jpg', gray_image)
cv2.imwrite('output/binaria.jpg', binary_image)

# --- Filtros: Blur, Gaussian Blur, Sharpen, Canny, Emboss ---
blurred_image = cv2.blur(image, (5, 5))
gaussian_blurred = cv2.GaussianBlur(image, (5, 5), 0)

sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
sharpened_image = cv2.filter2D(image, -1, sharpen_kernel)

edges = cv2.Canny(gray_image, 100, 200)

emboss_kernel = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
embossed_image = cv2.filter2D(image, -1, emboss_kernel)

# Guardar imágenes
cv2.imwrite('output/filters/blur.jpg', blurred_image)
cv2.imwrite('output/filters/gaussian_blur.jpg', gaussian_blurred)
cv2.imwrite('output/filters/sharpen.jpg', sharpened_image)
cv2.imwrite('output/filters/canny.jpg', edges)
cv2.imwrite('output/filters/emboss.jpg', embossed_image)

# --- Operaciones Morfológicas: Dilatación, Erosión, Apertura, Cierre ---
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(binary_image, kernel, iterations=1)
eroded = cv2.erode(binary_image, kernel, iterations=1)
opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

# Guardar imágenes
cv2.imwrite('output/morphologicalOperations/dilatada.jpg', dilated)
cv2.imwrite('output/morphologicalOperations/erosionada.jpg', eroded)
cv2.imwrite('output/morphologicalOperations/apertura.jpg', opening)
cv2.imwrite('output/morphologicalOperations/cierre.jpg', closing)

# --- Operaciones Aritméticas y Lógicas  ---
# Asumir una segunda imagen; reemplaza si es necesario
img2 = cv2.imread('assets/coronaCircular3d.jpg') 
sum_img = cv2.add(image, img2)
diff_img = cv2.subtract(image, img2)
mult_img = cv2.multiply(image, img2)
div_img = cv2.divide(image, img2)

_, img1_bin = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)
_, img2_bin = cv2.threshold(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY)
and_img = cv2.bitwise_and(img1_bin, img2_bin)
or_img = cv2.bitwise_or(img1_bin, img2_bin)
not_img = cv2.bitwise_not(img1_bin)

# Guardar imágenes
cv2.imwrite('output/arithmeticOperations/suma.jpg', sum_img)
cv2.imwrite('output/arithmeticOperations/resta.jpg', diff_img)
cv2.imwrite('output/arithmeticOperations/multiplicacion.jpg', mult_img)
cv2.imwrite('output/arithmeticOperations/division.jpg', div_img)
cv2.imwrite('output/arithmeticOperations/and.jpg', and_img)
cv2.imwrite('output/arithmeticOperations/or.jpg', or_img)
cv2.imwrite('output/arithmeticOperations/not.jpg', not_img)

# --- Transformaciones: Resize, Rotación, Interpolación ---
resized_bilinear = cv2.resize(image, (400, 400), interpolation=cv2.INTER_LINEAR)
resized_nearest = cv2.resize(image, (400, 400), interpolation=cv2.INTER_NEAREST)

(h, w) = image.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))

# Guardar imágenes
cv2.imwrite('output/transforms/resize_bilinear.jpg', resized_bilinear)
cv2.imwrite('output/transforms/resize_nearest.jpg', resized_nearest)
cv2.imwrite('output/transforms/rotada.jpg', rotated)

# --- Detección de Contornos y Formas (Ajustado para Anillos) ---
contours, hierarchy = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # Usar RETR_TREE para jerarquía con agujeros
contour_image = cv2.drawContours(np.copy(image), contours, -1, (0, 255, 0), 2)
cv2.imwrite('contornos.jpg', contour_image)

# Extraer características: Área, Perímetro, Forma, con detección de anillos
data = []
i = 0
while i < len(contours):
    contour = contours[i]
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # Detección de formas básicas
    approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
    sides = len(approx)
    shape = "Desconocida"
    is_ring = "No"
    ring_area = area
    ring_perimeter = perimeter
    
    if sides == 3:
        shape = "Triángulo"
    elif sides == 4:
        shape = "Cuadrado/Rectángulo"
    elif sides > 10:
        shape = "Círculo"
        circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
        
        # Verificar si es anillo: Si tiene un hijo (agujero) en la jerarquía
        if hierarchy is not None and hierarchy[0][i][2] != -1:  # Tiene hijo
            child_idx = hierarchy[0][i][2]
            child_contour = contours[child_idx]
            child_area = cv2.contourArea(child_contour)
            child_perimeter = cv2.arcLength(child_contour, True)
            child_approx = cv2.approxPolyDP(child_contour, 0.02 * child_perimeter, True)
            child_sides = len(child_approx)
            child_circularity = (4 * np.pi * child_area) / (child_perimeter ** 2) if child_perimeter > 0 else 0
            
            if child_sides > 10 and circularity > 0.8 and child_circularity > 0.8:
                shape = "Anillo (Corona Circular)"
                is_ring = "Sí"
                ring_area = area - child_area  # Área del anillo
                ring_perimeter = perimeter + child_perimeter  # Perímetro total
                i += 1  # Saltar el hijo en el bucle (ya procesado)
    
    data.append({"Objeto": len(data)+1, "Forma": shape, "Es Anillo": is_ring, "Área": ring_area, "Perímetro": ring_perimeter})
    i += 1

# Guardar datos en Excel
df = pd.DataFrame(data)
df.to_excel('resultados_formas.xlsx', index=False)

# --- Visualización ---
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title('Contornos y Formas (con Anillos)')
plt.axis('off')
plt.show()

print("Datos guardados en 'resultados_formas.xlsx'")
print(df)

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:665: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
